# pip는 최초 1회 환경 구성할때만(colab에선 매번 해줘야함)

In [ ]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install torch
!pip install transformers==4.8.2

     |████████████████████████████████| 344 kB 4.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=a23935ace1cc2572b4dbf6e76853b81160bc0f2d7df37a29b12182a732a75a4c
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 47.3 MB 1.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 1.1 MB 1.8 MB/s 
     |████████████████████████████████| 2.5 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 65.1 MB/s 
     |████████████████████████████████| 880 kB 58.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=127eff59b93b97ccde399afb365bb4afe497b670067150b658b85bdcde3ea7cf
  Stored in directory: /root/.

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-dthj55wk
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-dthj55wk
     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 4.5 MB 34.3 MB/s 
     |████████████████████████████████| 8.7 MB 77.4 MB/s 
     |████████████████████████████████| 79 kB 10.0 MB/s 
     |████████████████████████████████| 127 kB 78.4 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=180bd3402d1bf2db9dcc55ae1e7315f9728e82ab67043cb9fd1ebca784ff7d4c
  Stored in directory: /tmp/pip-ephem-wheel-cache-blgmngr_/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does no

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [ ]:
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/hug/ml/my_path/model.pth'))

<All keys matched successfully>

In [ ]:
def predict(predict_sentence):

    emotion = {0:'공포',1:'놀람',2:'분노',3:'슬픔',4:'중립',5:'행복',6:'혐오'}
    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits = i.detach().cpu().numpy()  
            test_eval.append(emotion[np.argmax(logits)])
        
        test_per=[]
        for i,logit in enumerate(logits):
            test_per.append(round((1/(1+np.exp((-1)*logit)))*100))
            print("{}:{}%".format(emotion[i],test_per[i]))
        print(">> 입력하신 내용에서 " + test_eval[0] + "가 느껴집니다.")

In [ ]:
while True :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : ㅇ


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


공포:11%
놀람:26%
분노:19%
슬픔:74%
중립:18%
행복:100%
혐오:39%
>> 입력하신 내용에서 행복가 느껴집니다.


하고싶은 말을 입력해주세요 : 부장님이 나에게 일을 너무 많이 시키신다.
공포:13%
놀람:10%
분노:100%
슬픔:42%
중립:29%
행복:24%
혐오:24%
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 네이버가 카카오보다 좋다
공포:35%
놀람:16%
분노:71%
슬픔:99%
중립:65%
행복:30%
혐오:6%
>> 입력하신 내용에서 슬픔가 느껴집니다.


하고싶은 말을 입력해주세요 : 참 좋겠다
공포:10%
놀람:7%
분노:79%
슬픔:66%
중립:77%
행복:69%
혐오:60%
>> 입력하신 내용에서 분노가 느껴집니다.




KeyboardInterrupt: ignored